<a href="https://colab.research.google.com/github/SattamAltwaim/SaSOKE/blob/main/notebooks/6_interactive_web_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤟 SOKE Interactive Web Service

**Real-time Text-to-Sign Language Generation with Interactive UI**

This notebook runs a web service that allows you to:
- Enter any text in a beautiful UI
- Generate sign language animations in real-time
- View 3D SMPL-X mesh animations interactively

### Requirements
- **GPU Runtime**: Go to `Runtime → Change runtime type → GPU` (T4 or better)
- **Google Drive**: Model files should be in your Drive (from notebook 1)


## 1. Setup Environment


In [1]:
# Clone repo if not present
import os
if not os.path.exists('/content/SaSOKE'):
    !git clone https://github.com/SattamAltwaim/SaSOKE.git
%cd /content/SaSOKE

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

drive_data = '/content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE'
print("✓ Code:", os.getcwd())
print("✓ Data:", drive_data)


Cloning into 'SaSOKE'...
remote: Enumerating objects: 542, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 542 (delta 27), reused 18 (delta 9), pack-reused 495 (from 1)
Receiving objects: 100% (542/542), 2.51 MiB | 4.93 MiB/s, done.
Resolving deltas: 100% (242/242), done.
/content/SaSOKE
Mounted at /content/drive
✓ Code: /content/SaSOKE
✓ Data: /content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE


In [2]:
# Install dependencies
%pip install -q pytorch_lightning torchmetrics omegaconf shortuuid transformers diffusers einops wandb rich matplotlib
%pip install -q smplx h5py scikit-image spacy ftfy more-itertools natsort tensorboard sentencepiece
%pip install -q fastapi uvicorn pyngrok python-multipart nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00


In [3]:
# Verify GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → GPU")


CUDA available: True
GPU: Tesla T4
GPU Memory: 15.83 GB


## 2. Setup Symlinks and Configuration


In [4]:
# Create symbolic links for deps
import os

deps_links = {
    'deps/smpl_models': f'{drive_data}/deps/smpl_models',
    'deps/mbart-h2s-csl-phoenix': f'{drive_data}/deps/mbart-h2s-csl-phoenix',
}

for expected_path, actual_path in deps_links.items():
    if not os.path.exists(expected_path):
        os.makedirs(os.path.dirname(expected_path), exist_ok=True)
        os.symlink(actual_path, expected_path)
        print(f"✓ {expected_path} linked")
    else:
        print(f"✓ {expected_path} already exists")

print("\n✓ All symbolic links created!")


✓ deps/smpl_models linked
✓ deps/mbart-h2s-csl-phoenix linked

✓ All symbolic links created!


In [5]:
# Configure paths
import sys
import yaml
from omegaconf import OmegaConf
from mGPT.config import parse_args

with open('configs/soke.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['ACCELERATOR'] = 'gpu'
config['DEVICE'] = [0]
config['DATASET']['H2S']['ROOT'] = f'{drive_data}/data/How2Sign'
config['DATASET']['H2S']['MEAN_PATH'] = f'{drive_data}/smpl-x/mean.pt'
config['DATASET']['H2S']['STD_PATH'] = f'{drive_data}/smpl-x/std.pt'
config['TRAIN']['PRETRAINED_VAE'] = f'{drive_data}/checkpoints/vae/tokenizer.ckpt'

with open('configs/web_inference.yaml', 'w') as f:
    yaml.dump(config, f)

# Update assets
with open('configs/assets.yaml', 'r') as f:
    assets = yaml.safe_load(f)

assets['RENDER']['SMPL_MODEL_PATH'] = 'deps/smpl_models/smpl'
assets['RENDER']['MODEL_PATH'] = 'deps/smpl_models'
assets['METRIC']['TM2T']['t2m_path'] = f'{drive_data}/deps/deps/t2m/t2m/'

with open('configs/assets_web.yaml', 'w') as f:
    yaml.dump(assets, f)

sys.argv = ['', '--cfg', 'configs/web_inference.yaml', '--cfg_assets', 'configs/assets_web.yaml']
cfg = parse_args(phase="test")
cfg.FOLDER = cfg.TEST.FOLDER

print("✓ Configuration ready!")


Force no debugging when testing
✓ Configuration ready!


## 3. Load Model


In [6]:
import torch
import pytorch_lightning as pl
from mGPT.models.build_model import build_model
from mGPT.data.build_data import build_data
from mGPT.utils.load_checkpoint import load_pretrained_vae, load_pretrained
from mGPT.utils.logger import create_logger

pl.seed_everything(cfg.SEED_VALUE)

cfg.DATASET.WORD_VERTILIZER_PATH = f'{drive_data}/deps/deps/t2m/glove/'

print("Loading model...")
datamodule = build_data(cfg)
model = build_model(cfg, datamodule)

logger = create_logger(cfg, phase="test")
if cfg.TRAIN.PRETRAINED_VAE:
    load_pretrained_vae(cfg, model, logger)

ckpt_path = f'{drive_data}/experiments/mgpt/SOKE/checkpoints/last.ckpt'
if os.path.exists(ckpt_path):
    print(f"Loading trained checkpoint from {ckpt_path}")
    cfg.TEST.CHECKPOINTS = ckpt_path
    load_pretrained(cfg, model, logger, phase="test")
else:
    print("Using pretrained mBART (no fine-tuned checkpoint found)")

model = model.cuda()
model.eval()

mean = datamodule.hparams.mean.cuda()
std = datamodule.hparams.std.cuda()

print("\n✓ Model ready!")


INFO:lightning_fabric.utilities.seed:Seed set to 1234


Loading model...
mean path /content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE/smpl-x/mean.pt std_path:  /content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE/smpl-x/std.pt


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
INFO:root:Loading pretrain vae from /content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE/checkpoints/vae/tokenizer.ckpt
2025-12-02 23:25:03,877 Loading pretrain vae from /content/drive/MyDrive/GraduationProject/CodeFiles/SaSOKE/checkpoints/vae/tokenizer.ckpt


load rhand vae...

=======Check Weights Loading======
Weights not used from pretrained file:
---------------------------
Weights not loaded into new model:

load hand vae...

=======Check Weights Loading======
Weights not used from pretrained file:
---------------------------
Weights not loaded into new model:

load vae...

=======Check Weights Loading======
Weights not used from pretrained file:
---------------------------
Weights not loaded into new model:

Using pretrained mBART (no fine-tuned checkpoint found)

✓ Model ready!


In [7]:
# Load SMPL-X for mesh generation
from mGPT.utils.human_models import smpl_x, get_coord

print(f"✓ SMPL-X model loaded")
print(f"  - Vertices: {smpl_x.vertex_num}")
print(f"  - Faces: {len(smpl_x.face)}")


✓ SMPL-X model loaded
  - Vertices: 10475
  - Faces: 20908


## 4. Start Interactive Web Service


NGROK SETUP (for public URL)

To expose the service publicly, you need an ngrok authtoken.
1. Sign up free at https://ngrok.com/
2. Get your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
3. Paste it below (or press Enter to skip)

Enter ngrok authtoken (or press Enter to skip): ··········
⚠ Skipping ngrok - will use Colab's built-in URL


In [9]:
# Define helper functions for the API
import time
import json
import numpy as np

def feats_to_smplx_api(features, mean_tensor, std_tensor):
    """Convert features to SMPL-X parameters for API response."""
    features = features * std_tensor + mean_tensor
    T = features.shape[0]
    zero_pose = torch.zeros(T, 36).to(features)
    features_full = torch.cat([zero_pose, features], dim=-1)

    return {
        'root_pose': features_full[:, 0:3].cpu().numpy().tolist(),
        'body_pose': features_full[:, 3:66].cpu().numpy().tolist(),
        'lhand_pose': features_full[:, 66:111].cpu().numpy().tolist(),
        'rhand_pose': features_full[:, 111:156].cpu().numpy().tolist(),
        'jaw_pose': features_full[:, 156:159].cpu().numpy().tolist(),
        'expression': features_full[:, 159:169].cpu().numpy().tolist(),
    }

def generate_mesh_api(smplx_params):
    """Generate mesh vertices for all frames."""
    num_frames = len(smplx_params['body_pose'])
    all_vertices = []

    shape_param = torch.tensor([[-0.07284723, 0.1795129, -0.27608207, 0.135155, 0.10748172,
                                 0.16037364, -0.01616933, -0.03450319, 0.01369138, 0.01108842]]).float()

    for i in range(num_frames):
        root_pose = torch.tensor([smplx_params['root_pose'][i]]).float()
        body_pose = torch.tensor([smplx_params['body_pose'][i]]).float()
        lhand_pose = torch.tensor([smplx_params['lhand_pose'][i]]).float()
        rhand_pose = torch.tensor([smplx_params['rhand_pose'][i]]).float()
        jaw_pose = torch.tensor([smplx_params['jaw_pose'][i]]).float()
        expression = torch.tensor([smplx_params['expression'][i]]).float()

        with torch.no_grad():
            vertices, _ = get_coord(
                root_pose=root_pose,
                body_pose=body_pose,
                lhand_pose=lhand_pose,
                rhand_pose=rhand_pose,
                jaw_pose=jaw_pose,
                shape=shape_param,
                expr=expression
            )

        all_vertices.append(vertices[0].cpu().numpy().tolist())

    return {
        "vertices": all_vertices,
        "faces": smpl_x.face.tolist()
    }

print("✓ Helper functions defined")


✓ Helper functions defined


In [10]:
# Create FastAPI application
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from typing import Optional

app = FastAPI(title="SOKE Text-to-Sign API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TextRequest(BaseModel):
    text: str
    sign_language: str = "how2sign"
    fps: int = 20
    include_mesh: bool = True

# Note: The root route "/" is defined in the next cell after FRONTEND_HTML is created

@app.get("/health")
async def health():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "model_loaded": True,
        "gpu_available": torch.cuda.is_available()
    }

@app.get("/languages")
async def languages():
    """Get supported sign languages."""
    return {
        "languages": [
            {"id": "how2sign", "name": "American Sign Language (ASL)", "input_language": "English"},
            {"id": "csl", "name": "Chinese Sign Language (CSL)", "input_language": "Chinese"},
            {"id": "phoenix", "name": "German Sign Language (DGS)", "input_language": "German"}
        ]
    }

@app.post("/generate")
async def generate(request: TextRequest):
    """Generate sign language from text."""
    start_time = time.time()

    try:
        batch = {
            'text': [request.text],
            'length': [0],
            'src': [request.sign_language]
        }

        with torch.no_grad():
            output = model.forward(batch, task="t2m")

        feats = output['feats'][0] if 'feats' in output else None

        if feats is None:
            return {"success": False, "error": "No features generated", "text": request.text, "num_frames": 0, "fps": request.fps, "generation_time": time.time() - start_time}

        smplx_params = feats_to_smplx_api(feats, mean, std)
        num_frames = len(smplx_params['body_pose'])

        mesh_data = None
        if request.include_mesh:
            mesh_data = generate_mesh_api(smplx_params)

        return {
            "success": True,
            "text": request.text,
            "num_frames": num_frames,
            "fps": request.fps,
            "smplx_params": smplx_params,
            "mesh_data": mesh_data,
            "generation_time": time.time() - start_time
        }

    except Exception as e:
        import traceback
        traceback.print_exc()
        return {"success": False, "error": str(e), "text": request.text, "num_frames": 0, "fps": request.fps, "generation_time": time.time() - start_time}

print("✓ FastAPI app created")


✓ FastAPI app created


In [11]:
# Define the interactive frontend HTML
FRONTEND_HTML = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SOKE - Text to Sign Language</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Outfit:wght@300;400;500;600;700&family=JetBrains+Mono:wght@400;500&display=swap" rel="stylesheet">
    <style>
        :root { --bg-dark: #0a0a0f; --bg-surface: #12121a; --bg-card: #1a1a24; --accent-primary: #6366f1; --accent-secondary: #8b5cf6; --accent-glow: rgba(99, 102, 241, 0.4); --text-primary: #f1f5f9; --text-secondary: #94a3b8; --text-muted: #64748b; --border-color: rgba(255, 255, 255, 0.08); --success: #22c55e; --error: #ef4444; }
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Outfit', sans-serif; background: var(--bg-dark); color: var(--text-primary); min-height: 100vh; }
        .bg-pattern { position: fixed; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; z-index: 0; background: radial-gradient(ellipse at 20% 20%, rgba(99, 102, 241, 0.08) 0%, transparent 50%), radial-gradient(ellipse at 80% 80%, rgba(139, 92, 246, 0.08) 0%, transparent 50%); }
        .container { position: relative; z-index: 1; max-width: 1400px; margin: 0 auto; padding: 20px; min-height: 100vh; display: flex; flex-direction: column; }
        header { text-align: center; padding: 30px 0 20px; }
        .logo { display: inline-flex; align-items: center; gap: 12px; margin-bottom: 10px; }
        .logo-icon { width: 48px; height: 48px; background: linear-gradient(135deg, var(--accent-primary), var(--accent-secondary)); border-radius: 14px; display: flex; align-items: center; justify-content: center; font-size: 24px; box-shadow: 0 8px 32px var(--accent-glow); }
        h1 { font-size: 2rem; font-weight: 700; background: linear-gradient(135deg, var(--text-primary), var(--accent-primary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }
        .subtitle { color: var(--text-secondary); font-size: 1rem; }
        .main-layout { display: grid; grid-template-columns: 380px 1fr; gap: 20px; flex: 1; }
        .panel { background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 16px; overflow: hidden; }
        .panel-header { padding: 16px 20px; border-bottom: 1px solid var(--border-color); display: flex; align-items: center; gap: 10px; }
        .panel-header h2 { font-size: 0.9rem; font-weight: 600; }
        .panel-icon { width: 32px; height: 32px; background: linear-gradient(135deg, rgba(99, 102, 241, 0.2), rgba(139, 92, 246, 0.2)); border-radius: 8px; display: flex; align-items: center; justify-content: center; font-size: 16px; }
        .panel-body { padding: 20px; }
        .form-group { margin-bottom: 16px; }
        .form-group label { display: block; font-size: 0.8rem; font-weight: 500; color: var(--text-secondary); margin-bottom: 6px; }
        textarea { width: 100%; min-height: 100px; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 10px; padding: 12px; font-family: 'Outfit', sans-serif; font-size: 0.95rem; color: var(--text-primary); resize: vertical; transition: all 0.2s; }
        textarea:focus { outline: none; border-color: var(--accent-primary); box-shadow: 0 0 0 3px var(--accent-glow); }
        textarea::placeholder { color: var(--text-muted); }
        select { width: 100%; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 10px; padding: 12px; font-family: 'Outfit', sans-serif; font-size: 0.95rem; color: var(--text-primary); cursor: pointer; appearance: none; background-image: url("data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' width='24' height='24' viewBox='0 0 24 24' fill='none' stroke='%2394a3b8' stroke-width='2'%3E%3Cpolyline points='6 9 12 15 18 9'%3E%3C/polyline%3E%3C/svg%3E"); background-repeat: no-repeat; background-position: right 10px center; background-size: 16px; }
        select:focus { outline: none; border-color: var(--accent-primary); }
        .generate-btn { width: 100%; padding: 14px 20px; background: linear-gradient(135deg, var(--accent-primary), var(--accent-secondary)); border: none; border-radius: 12px; font-family: 'Outfit', sans-serif; font-size: 1rem; font-weight: 600; color: white; cursor: pointer; transition: all 0.3s; display: flex; align-items: center; justify-content: center; gap: 8px; box-shadow: 0 4px 20px var(--accent-glow); }
        .generate-btn:hover:not(:disabled) { transform: translateY(-2px); box-shadow: 0 8px 30px var(--accent-glow); }
        .generate-btn:disabled { opacity: 0.6; cursor: not-allowed; }
        .generate-btn .spinner { width: 18px; height: 18px; border: 2px solid rgba(255,255,255,0.3); border-top-color: white; border-radius: 50%; animation: spin 0.8s linear infinite; }
        @keyframes spin { to { transform: rotate(360deg); } }
        .status-bar { margin-top: 12px; padding: 10px 12px; border-radius: 10px; font-size: 0.8rem; display: none; align-items: center; gap: 8px; }
        .status-bar.show { display: flex; }
        .status-bar.info { background: rgba(99, 102, 241, 0.1); border: 1px solid rgba(99, 102, 241, 0.2); color: var(--accent-primary); }
        .status-bar.success { background: rgba(34, 197, 94, 0.1); border: 1px solid rgba(34, 197, 94, 0.2); color: var(--success); }
        .status-bar.error { background: rgba(239, 68, 68, 0.1); border: 1px solid rgba(239, 68, 68, 0.2); color: var(--error); }
        #viewer-container { flex: 1; min-height: 400px; background: var(--bg-surface); border-radius: 12px; overflow: hidden; position: relative; }
        #three-canvas { width: 100%; height: 100%; display: block; }
        .viewer-placeholder { position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); text-align: center; color: var(--text-muted); }
        .viewer-placeholder .icon { font-size: 48px; margin-bottom: 12px; opacity: 0.5; }
        .playback-controls { display: flex; align-items: center; gap: 10px; padding: 16px 20px; background: var(--bg-surface); border-top: 1px solid var(--border-color); }
        .control-btn { width: 40px; height: 40px; background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 10px; color: var(--text-primary); font-size: 16px; cursor: pointer; transition: all 0.2s; display: flex; align-items: center; justify-content: center; }
        .control-btn:hover:not(:disabled) { background: var(--accent-primary); border-color: var(--accent-primary); }
        .control-btn:disabled { opacity: 0.4; cursor: not-allowed; }
        .timeline { flex: 1; height: 6px; background: var(--bg-card); border-radius: 3px; cursor: pointer; overflow: hidden; }
        .timeline-progress { height: 100%; background: linear-gradient(90deg, var(--accent-primary), var(--accent-secondary)); border-radius: 3px; width: 0%; }
        .frame-counter { font-family: 'JetBrains Mono', monospace; font-size: 0.8rem; color: var(--text-secondary); min-width: 80px; text-align: right; }
        .speed-select { background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 6px; padding: 6px 10px; font-family: 'JetBrains Mono', monospace; font-size: 0.7rem; color: var(--text-primary); }
        .stats-grid { display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; margin-top: 16px; display: none; }
        .stats-grid.show { display: grid; }
        .stat-item { background: var(--bg-surface); border-radius: 10px; padding: 12px; text-align: center; }
        .stat-value { font-family: 'JetBrains Mono', monospace; font-size: 1.1rem; font-weight: 600; color: var(--accent-primary); }
        .stat-label { font-size: 0.7rem; color: var(--text-muted); margin-top: 2px; }
        .examples { margin-top: 16px; padding-top: 16px; border-top: 1px solid var(--border-color); }
        .examples h3 { font-size: 0.8rem; font-weight: 500; color: var(--text-secondary); margin-bottom: 10px; }
        .example-chips { display: flex; flex-wrap: wrap; gap: 6px; }
        .example-chip { padding: 6px 12px; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 16px; font-size: 0.75rem; color: var(--text-secondary); cursor: pointer; transition: all 0.2s; }
        .example-chip:hover { background: rgba(99, 102, 241, 0.1); border-color: var(--accent-primary); color: var(--accent-primary); }
        .loading-overlay { position: absolute; top: 0; left: 0; width: 100%; height: 100%; background: rgba(10, 10, 15, 0.9); display: flex; flex-direction: column; align-items: center; justify-content: center; z-index: 100; opacity: 0; pointer-events: none; transition: opacity 0.3s; }
        .loading-overlay.active { opacity: 1; pointer-events: all; }
        .loading-spinner { width: 40px; height: 40px; border: 3px solid var(--border-color); border-top-color: var(--accent-primary); border-radius: 50%; animation: spin 1s linear infinite; }
        .loading-text { margin-top: 12px; color: var(--text-secondary); font-size: 0.85rem; }
        @media (max-width: 900px) { .main-layout { grid-template-columns: 1fr; } #viewer-container { min-height: 350px; } }
    </style>
</head>
<body>
    <div class="bg-pattern"></div>
    <div class="container">
        <header>
            <div class="logo"><div class="logo-icon">🤟</div><h1>SOKE</h1></div>
            <p class="subtitle">Real-time Text to Sign Language Generation</p>
        </header>
        <div class="main-layout">
            <div class="panel">
                <div class="panel-header"><div class="panel-icon">✍️</div><h2>Input</h2></div>
                <div class="panel-body">
                    <div class="form-group">
                        <label for="text-input">Enter text to translate</label>
                        <textarea id="text-input" placeholder="Type a sentence...">Hello, how are you today?</textarea>
                    </div>
                    <div class="form-group">
                        <label for="language-select">Target Sign Language</label>
                        <select id="language-select">
                            <option value="how2sign">🇺🇸 American Sign Language (ASL)</option>
                            <option value="csl">🇨🇳 Chinese Sign Language (CSL)</option>
                            <option value="phoenix">🇩🇪 German Sign Language (DGS)</option>
                        </select>
                    </div>
                    <button id="generate-btn" class="generate-btn"><span class="btn-text">🚀 Generate Sign Language</span></button>
                    <div id="status-bar" class="status-bar"><span class="status-text"></span></div>
                    <div class="examples">
                        <h3>Try these examples:</h3>
                        <div class="example-chips">
                            <span class="example-chip" data-text="Hello, how are you?">Hello</span>
                            <span class="example-chip" data-text="Thank you for your help.">Thank you</span>
                            <span class="example-chip" data-text="Nice to meet you!">Nice to meet you</span>
                            <span class="example-chip" data-text="What is your name?">What's your name?</span>
                        </div>
                    </div>
                    <div class="stats-grid" id="stats-grid">
                        <div class="stat-item"><div class="stat-value" id="stat-frames">-</div><div class="stat-label">Frames</div></div>
                        <div class="stat-item"><div class="stat-value" id="stat-duration">-</div><div class="stat-label">Duration</div></div>
                        <div class="stat-item"><div class="stat-value" id="stat-time">-</div><div class="stat-label">Gen Time</div></div>
                    </div>
                </div>
            </div>
            <div class="panel" style="display: flex; flex-direction: column;">
                <div class="panel-header"><div class="panel-icon">👁️</div><h2>3D Visualization</h2></div>
                <div class="panel-body" style="padding: 0; flex: 1; display: flex; flex-direction: column;">
                    <div id="viewer-container">
                        <canvas id="three-canvas"></canvas>
                        <div class="viewer-placeholder" id="placeholder"><div class="icon">🧍</div><p>Enter text and click Generate</p></div>
                        <div class="loading-overlay" id="loading-overlay"><div class="loading-spinner"></div><div class="loading-text">Generating...</div></div>
                    </div>
                    <div class="playback-controls">
                        <button class="control-btn" id="play-btn" disabled>▶</button>
                        <button class="control-btn" id="pause-btn" disabled>⏸</button>
                        <button class="control-btn" id="reset-btn" disabled>↺</button>
                        <div class="timeline" id="timeline"><div class="timeline-progress" id="timeline-progress"></div></div>
                        <span class="frame-counter" id="frame-counter">0 / 0</span>
                        <select class="speed-select" id="speed-select"><option value="0.5">0.5x</option><option value="1" selected>1x</option><option value="2">2x</option></select>
                    </div>
                </div>
            </div>
        </div>
    </div>
    <script>
        const API_URL = window.location.origin;
        const FPS = 20;
        let meshData = null, currentFrame = 0, totalFrames = 0, isPlaying = false, animationId = null, lastFrameTime = 0, playbackSpeed = 1;
        let scene, camera, renderer, controls, mesh = null, geometry = null;

        function initThreeJS() {
            const container = document.getElementById('viewer-container');
            const canvas = document.getElementById('three-canvas');
            scene = new THREE.Scene();
            scene.background = new THREE.Color(0x12121a);
            camera = new THREE.PerspectiveCamera(45, container.clientWidth / container.clientHeight, 0.1, 100);
            camera.position.set(0, 0.5, 2.5);
            renderer = new THREE.WebGLRenderer({ canvas, antialias: true });
            renderer.setSize(container.clientWidth, container.clientHeight);
            renderer.setPixelRatio(window.devicePixelRatio);
            controls = new THREE.OrbitControls(camera, renderer.domElement);
            controls.enableDamping = true;
            controls.target.set(0, 0.5, 0);
            scene.add(new THREE.AmbientLight(0xffffff, 0.5));
            const light = new THREE.DirectionalLight(0xffffff, 0.8);
            light.position.set(2, 3, 2);
            scene.add(light);
            scene.add(new THREE.DirectionalLight(0x6366f1, 0.3).position.set(-2, 1, -1));
            const grid = new THREE.GridHelper(4, 20, 0x333344, 0x222233);
            grid.position.y = -0.5;
            scene.add(grid);
            window.addEventListener('resize', () => {
                camera.aspect = container.clientWidth / container.clientHeight;
                camera.updateProjectionMatrix();
                renderer.setSize(container.clientWidth, container.clientHeight);
            });
            (function animate() { requestAnimationFrame(animate); controls.update(); renderer.render(scene, camera); })();
        }

        function createMesh(vertices, faces) {
            if (mesh) { scene.remove(mesh); geometry.dispose(); }
            geometry = new THREE.BufferGeometry();
            geometry.setAttribute('position', new THREE.BufferAttribute(new Float32Array(vertices.flat()), 3));
            geometry.setIndex(new THREE.BufferAttribute(new Uint32Array(faces.flat()), 1));
            geometry.computeVertexNormals();
            mesh = new THREE.Mesh(geometry, new THREE.MeshPhongMaterial({ color: 0x6366f1, shininess: 30, side: THREE.DoubleSide }));
            scene.add(mesh);
            const box = new THREE.Box3().setFromObject(mesh);
            controls.target.copy(box.getCenter(new THREE.Vector3()));
        }

        function updateMesh(vertices) {
            if (!geometry) return;
            geometry.setAttribute('position', new THREE.BufferAttribute(new Float32Array(vertices.flat()), 3));
            geometry.computeVertexNormals();
        }

        function playAnimation() {
            if (!meshData || totalFrames === 0) return;
            isPlaying = true;
            document.getElementById('play-btn').disabled = true;
            document.getElementById('pause-btn').disabled = false;
            lastFrameTime = performance.now();
            function step(ts) {
                if (!isPlaying) return;
                if (ts - lastFrameTime >= 1000 / (FPS * playbackSpeed)) {
                    currentFrame = (currentFrame + 1) % totalFrames;
                    updateMesh(meshData.vertices[currentFrame]);
                    updateUI();
                    lastFrameTime = ts;
                }
                animationId = requestAnimationFrame(step);
            }
            animationId = requestAnimationFrame(step);
        }

        function pauseAnimation() {
            isPlaying = false;
            document.getElementById('play-btn').disabled = false;
            document.getElementById('pause-btn').disabled = true;
            if (animationId) cancelAnimationFrame(animationId);
        }

        function resetAnimation() {
            pauseAnimation();
            currentFrame = 0;
            if (meshData) updateMesh(meshData.vertices[0]);
            updateUI();
        }

        function updateUI() {
            document.getElementById('frame-counter').textContent = `${currentFrame + 1} / ${totalFrames}`;
            document.getElementById('timeline-progress').style.width = `${((currentFrame + 1) / totalFrames) * 100}%`;
        }

        function showStatus(type, msg) {
            const bar = document.getElementById('status-bar');
            bar.className = `status-bar show ${type}`;
            bar.querySelector('.status-text').textContent = msg;
            if (type === 'success') setTimeout(() => bar.classList.remove('show'), 4000);
        }

        async function generate() {
            const text = document.getElementById('text-input').value.trim();
            if (!text) { showStatus('error', 'Please enter text'); return; }
            const btn = document.getElementById('generate-btn');
            btn.disabled = true;
            btn.innerHTML = '<div class="spinner"></div><span>Generating...</span>';
            document.getElementById('placeholder').style.display = 'none';
            document.getElementById('loading-overlay').classList.add('active');
            showStatus('info', 'Generating sign language...');
            try {
                const res = await fetch(`${API_URL}/generate`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ text, sign_language: document.getElementById('language-select').value, fps: FPS, include_mesh: true })
                });
                const data = await res.json();
                if (!data.success) throw new Error(data.error || 'Failed');
                if (data.mesh_data?.vertices && data.mesh_data?.faces) {
                    meshData = data.mesh_data;
                    totalFrames = meshData.vertices.length;
                    currentFrame = 0;
                    createMesh(meshData.vertices[0], meshData.faces);
                    document.getElementById('play-btn').disabled = false;
                    document.getElementById('reset-btn').disabled = false;
                    document.getElementById('stats-grid').classList.add('show');
                    document.getElementById('stat-frames').textContent = data.num_frames;
                    document.getElementById('stat-duration').textContent = `${(data.num_frames / FPS).toFixed(2)}s`;
                    document.getElementById('stat-time').textContent = `${data.generation_time.toFixed(2)}s`;
                    updateUI();
                    showStatus('success', `Generated ${data.num_frames} frames!`);
                    playAnimation();
                } else throw new Error('No mesh data');
            } catch (e) {
                showStatus('error', `Error: ${e.message}`);
                document.getElementById('placeholder').style.display = 'flex';
            } finally {
                btn.disabled = false;
                btn.innerHTML = '<span class="btn-text">🚀 Generate Sign Language</span>';
                document.getElementById('loading-overlay').classList.remove('active');
            }
        }

        document.addEventListener('DOMContentLoaded', () => {
            initThreeJS();
            document.getElementById('generate-btn').addEventListener('click', generate);
            document.getElementById('play-btn').addEventListener('click', playAnimation);
            document.getElementById('pause-btn').addEventListener('click', pauseAnimation);
            document.getElementById('reset-btn').addEventListener('click', resetAnimation);
            document.getElementById('timeline').addEventListener('click', e => {
                pauseAnimation();
                currentFrame = Math.floor((e.clientX - e.target.getBoundingClientRect().left) / e.target.offsetWidth * totalFrames);
                if (meshData) updateMesh(meshData.vertices[currentFrame]);
                updateUI();
            });
            document.getElementById('speed-select').addEventListener('change', e => { playbackSpeed = parseFloat(e.target.value); });
            document.querySelectorAll('.example-chip').forEach(c => c.addEventListener('click', () => { document.getElementById('text-input').value = c.dataset.text; }));
            document.getElementById('text-input').addEventListener('keydown', e => { if (e.key === 'Enter' && e.ctrlKey) generate(); });
            fetch(`${API_URL}/health`).then(r => r.json()).then(d => { if (d.status === 'healthy') showStatus('success', 'Connected to SOKE API ✓'); }).catch(() => showStatus('info', 'Connecting...'));
        });
    </script>
</body>
</html>'''

print("✓ Frontend HTML defined")


✓ Frontend HTML defined


In [12]:
# Configure the root route to serve the frontend
from fastapi.responses import HTMLResponse

@app.get("/", response_class=HTMLResponse)
async def serve_ui():
    """Serve the interactive frontend UI."""
    return FRONTEND_HTML

print("✓ Frontend route configured - UI will be served at /")


✓ Frontend route configured - UI will be served at /


In [13]:
# Check prerequisites before starting server
print("=" * 60)
print("CHECKING PREREQUISITES")
print("=" * 60)

errors = []

# Check if model is loaded
try:
    _ = model
    print("✓ Model loaded")
except NameError:
    errors.append("Model not loaded - run cells 3-10 first")
    print("❌ Model not loaded")

# Check if mean/std are loaded
try:
    _ = mean
    _ = std
    print("✓ Mean/std loaded")
except NameError:
    errors.append("Mean/std not loaded - run cell 10")
    print("❌ Mean/std not loaded")

# Check if SMPL-X is loaded
try:
    _ = smpl_x
    _ = get_coord
    print("✓ SMPL-X loaded")
except NameError:
    errors.append("SMPL-X not loaded - run cell 11")
    print("❌ SMPL-X not loaded")

# Check if helper functions are defined
try:
    _ = feats_to_smplx_api
    _ = generate_mesh_api
    print("✓ Helper functions defined")
except NameError:
    errors.append("Helper functions not defined - run cell 14")
    print("❌ Helper functions not defined")

# Check if FastAPI app is created
try:
    _ = app
    print("✓ FastAPI app created")
except NameError:
    errors.append("FastAPI app not created - run cell 15")
    print("❌ FastAPI app not created")

# Check if FRONTEND_HTML is defined
try:
    _ = FRONTEND_HTML
    print("✓ Frontend HTML defined")
except NameError:
    errors.append("Frontend HTML not defined - run cell 16")
    print("❌ Frontend HTML not defined")

if errors:
    print("\n" + "=" * 60)
    print("❌ CANNOT START SERVER - MISSING PREREQUISITES:")
    print("=" * 60)
    for err in errors:
        print(f"   • {err}")
    print("\n⚠️  Please run all cells from the beginning (cells 3-16)")
else:
    print("\n✅ All prerequisites OK! Starting server...")

    import nest_asyncio
    import uvicorn
    from threading import Thread
    import socket

    nest_asyncio.apply()

    # Find an available port
    def find_free_port(start_port=8080):
        for port in range(start_port, start_port + 100):
            try:
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                    s.bind(('0.0.0.0', port))
                    return port
            except OSError:
                continue
        return None

    PORT = find_free_port(8080)

    if PORT is None:
        print("❌ No available ports found!")
        print("   Try: Runtime → Restart runtime")
    else:
        print(f"Using port {PORT}")

        def run_server():
            try:
                uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="warning")
            except Exception as e:
                print(f"❌ Server error: {e}")

        server_thread = Thread(target=run_server, daemon=True)
        server_thread.start()

        import time
        time.sleep(3)

        # Test server
        import requests
        try:
            resp = requests.get(f"http://localhost:{PORT}/health", timeout=5)
            if resp.status_code == 200:
                print(f"\n✅ SERVER IS RUNNING on port {PORT}!")
                print(f"\n📱 Run the NEXT CELL to get your Colab URL")
            else:
                print(f"❌ Server returned status {resp.status_code}")
        except Exception as e:
            print(f"❌ Server failed to start: {e}")
            print("   Try: Runtime → Restart runtime, then run ALL cells again")


CHECKING PREREQUISITES
✓ Model loaded
✓ Mean/std loaded
✓ SMPL-X loaded
✓ Helper functions defined
✓ FastAPI app created
✓ Frontend HTML defined

✅ All prerequisites OK! Starting server...
Using port 8081

✅ SERVER IS RUNNING on port 8081!

📱 Run the NEXT CELL to get your Colab URL


In [14]:
# DIAGNOSTIC - Check what's actually running
try:
    PORT
except NameError:
    PORT = 8080

import requests
from google.colab.output import eval_js

print("=" * 60)
print("DIAGNOSTIC CHECK")
print("=" * 60)

# Check if server is responding locally
try:
    resp = requests.get(f"http://localhost:{PORT}/health", timeout=3)
    if resp.status_code == 200:
        print(f"✅ Server is running on port {PORT}")

        # Try to get the homepage
        try:
            home = requests.get(f"http://localhost:{PORT}/", timeout=3)
            if home.status_code == 200 and len(home.text) > 1000:
                print(f"✅ Homepage is being served ({len(home.text)} bytes)")
            else:
                print(f"⚠️  Homepage returned status {home.status_code}")
        except Exception as e:
            print(f"❌ Cannot access homepage: {e}")

        # Get the Colab URL
        try:
            colab_url = eval_js(f'google.colab.kernel.proxyPort({PORT})')
            print(f"\n{'=' * 60}")
            print(f"🌐 YOUR WORKING URL:")
            print(f"{'=' * 60}")
            print(f"\n{colab_url}")
            print(f"\n{'=' * 60}")
            print(f"\n📱 Copy and paste this URL into a NEW browser tab")
            print(f"   (Don't use the old URL - it's from a previous session)")
        except Exception as e:
            print(f"❌ Could not get Colab URL: {e}")

    else:
        print(f"❌ Server returned status {resp.status_code}")
        print(f"   Re-run cells 15-18 to start the server")

except requests.exceptions.ConnectionError:
    print(f"❌ No server running on port {PORT}")
    print(f"   You need to run cells 15-18 first!")
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Make sure you ran cells 15-18")


DIAGNOSTIC CHECK
✅ Server is running on port 8081
✅ Homepage is being served (22442 bytes)

🌐 YOUR WORKING URL:

https://8081-gpu-t4-s-1u7ewy4rsqj7d-c.asia-southeast1-0.prod.colab.dev


📱 Copy and paste this URL into a NEW browser tab
   (Don't use the old URL - it's from a previous session)


Testing server routes...
✓ /health: 200
✓ /: 200 - Content length: 22442 bytes
  ✅ Homepage is working!


Removed old route: /
✅ UI route registered!

🌐 Now visit: https://8082-gpu-t4-s-1k12xo2lz52wo-c.asia-southeast1-2.prod.colab.dev
   (Refresh the page if it's already open)


## 🎯 Generate Standalone UI File (RECOMMENDED)

**Download and open this HTML file locally - no ngrok needed!**


In [ ]:
# Generate standalone HTML file with your Colab server URL
try:
    PORT
except NameError:
    PORT = 8080

from google.colab.output import eval_js
import os

# Get current Colab URL
try:
    colab_url = eval_js(f'google.colab.kernel.proxyPort({PORT})')
    
    # Read the HTML template
    html_template = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SOKE - Text to Sign Language</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Outfit:wght@300;400;500;600;700&family=JetBrains+Mono:wght@400;500&display=swap" rel="stylesheet">
    <style>
        :root { --bg-dark: #0a0a0f; --bg-surface: #12121a; --bg-card: #1a1a24; --accent-primary: #6366f1; --accent-secondary: #8b5cf6; --accent-glow: rgba(99, 102, 241, 0.4); --text-primary: #f1f5f9; --text-secondary: #94a3b8; --text-muted: #64748b; --border-color: rgba(255, 255, 255, 0.08); --success: #22c55e; --error: #ef4444; }
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Outfit', sans-serif; background: var(--bg-dark); color: var(--text-primary); min-height: 100vh; }
        .bg-pattern { position: fixed; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; z-index: 0; background: radial-gradient(ellipse at 20% 20%, rgba(99, 102, 241, 0.08) 0%, transparent 50%), radial-gradient(ellipse at 80% 80%, rgba(139, 92, 246, 0.08) 0%, transparent 50%); }
        .container { position: relative; z-index: 1; max-width: 1400px; margin: 0 auto; padding: 20px; min-height: 100vh; display: flex; flex-direction: column; }
        header { text-align: center; padding: 30px 0 20px; }
        .logo { display: inline-flex; align-items: center; gap: 12px; margin-bottom: 10px; }
        .logo-icon { width: 48px; height: 48px; background: linear-gradient(135deg, var(--accent-primary), var(--accent-secondary)); border-radius: 14px; display: flex; align-items: center; justify-content: center; font-size: 24px; box-shadow: 0 8px 32px var(--accent-glow); }
        h1 { font-size: 2rem; font-weight: 700; background: linear-gradient(135deg, var(--text-primary), var(--accent-primary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }
        .subtitle { color: var(--text-secondary); font-size: 1rem; }
        .server-url-box { margin: 10px auto; padding: 12px 20px; background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 12px; max-width: 600px; }
        .server-url-label { font-size: 0.75rem; color: var(--text-muted); margin-bottom: 8px; }
        .server-url-input { width: 100%; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 8px; padding: 10px 12px; font-family: 'JetBrains Mono', monospace; font-size: 0.85rem; color: var(--text-primary); }
        .server-url-input:focus { outline: none; border-color: var(--accent-primary); }
        .server-status { display: inline-block; margin-left: 10px; padding: 4px 12px; border-radius: 12px; font-size: 0.75rem; font-weight: 600; }
        .server-status.connected { background: rgba(34, 197, 94, 0.2); color: var(--success); }
        .server-status.disconnected { background: rgba(239, 68, 68, 0.2); color: var(--error); }
        .main-layout { display: grid; grid-template-columns: 380px 1fr; gap: 20px; flex: 1; }
        .panel { background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 16px; overflow: hidden; }
        .panel-header { padding: 16px 20px; border-bottom: 1px solid var(--border-color); display: flex; align-items: center; gap: 10px; }
        .panel-header h2 { font-size: 0.9rem; font-weight: 600; }
        .panel-icon { width: 32px; height: 32px; background: linear-gradient(135deg, rgba(99, 102, 241, 0.2), rgba(139, 92, 246, 0.2)); border-radius: 8px; display: flex; align-items: center; justify-content: center; font-size: 16px; }
        .panel-body { padding: 20px; }
        .form-group { margin-bottom: 16px; }
        .form-group label { display: block; font-size: 0.8rem; font-weight: 500; color: var(--text-secondary); margin-bottom: 6px; }
        textarea { width: 100%; min-height: 100px; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 10px; padding: 12px; font-family: 'Outfit', sans-serif; font-size: 0.95rem; color: var(--text-primary); resize: vertical; transition: all 0.2s; }
        textarea:focus { outline: none; border-color: var(--accent-primary); box-shadow: 0 0 0 3px var(--accent-glow); }
        textarea::placeholder { color: var(--text-muted); }
        select { width: 100%; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 10px; padding: 12px; font-family: 'Outfit', sans-serif; font-size: 0.95rem; color: var(--text-primary); cursor: pointer; appearance: none; background-image: url("data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' width='24' height='24' viewBox='0 0 24 24' fill='none' stroke='%2394a3b8' stroke-width='2'%3E%3Cpolyline points='6 9 12 15 18 9'%3E%3C/polyline%3E%3C/svg%3E"); background-repeat: no-repeat; background-position: right 10px center; background-size: 16px; }
        select:focus { outline: none; border-color: var(--accent-primary); }
        .generate-btn { width: 100%; padding: 14px 20px; background: linear-gradient(135deg, var(--accent-primary), var(--accent-secondary)); border: none; border-radius: 12px; font-family: 'Outfit', sans-serif; font-size: 1rem; font-weight: 600; color: white; cursor: pointer; transition: all 0.3s; display: flex; align-items: center; justify-content: center; gap: 8px; box-shadow: 0 4px 20px var(--accent-glow); }
        .generate-btn:hover:not(:disabled) { transform: translateY(-2px); box-shadow: 0 8px 30px var(--accent-glow); }
        .generate-btn:disabled { opacity: 0.6; cursor: not-allowed; }
        .generate-btn .spinner { width: 18px; height: 18px; border: 2px solid rgba(255,255,255,0.3); border-top-color: white; border-radius: 50%; animation: spin 0.8s linear infinite; }
        @keyframes spin { to { transform: rotate(360deg); } }
        .status-bar { margin-top: 12px; padding: 10px 12px; border-radius: 10px; font-size: 0.8rem; display: none; align-items: center; gap: 8px; }
        .status-bar.show { display: flex; }
        .status-bar.info { background: rgba(99, 102, 241, 0.1); border: 1px solid rgba(99, 102, 241, 0.2); color: var(--accent-primary); }
        .status-bar.success { background: rgba(34, 197, 94, 0.1); border: 1px solid rgba(34, 197, 94, 0.2); color: var(--success); }
        .status-bar.error { background: rgba(239, 68, 68, 0.1); border: 1px solid rgba(239, 68, 68, 0.2); color: var(--error); }
        #viewer-container { flex: 1; min-height: 400px; background: var(--bg-surface); border-radius: 12px; overflow: hidden; position: relative; }
        #three-canvas { width: 100%; height: 100%; display: block; }
        .viewer-placeholder { position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); text-align: center; color: var(--text-muted); }
        .viewer-placeholder .icon { font-size: 48px; margin-bottom: 12px; opacity: 0.5; }
        .playback-controls { display: flex; align-items: center; gap: 10px; padding: 16px 20px; background: var(--bg-surface); border-top: 1px solid var(--border-color); }
        .control-btn { width: 40px; height: 40px; background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 10px; color: var(--text-primary); font-size: 16px; cursor: pointer; transition: all 0.2s; display: flex; align-items: center; justify-content: center; }
        .control-btn:hover:not(:disabled) { background: var(--accent-primary); border-color: var(--accent-primary); }
        .control-btn:disabled { opacity: 0.4; cursor: not-allowed; }
        .timeline { flex: 1; height: 6px; background: var(--bg-card); border-radius: 3px; cursor: pointer; overflow: hidden; }
        .timeline-progress { height: 100%; background: linear-gradient(90deg, var(--accent-primary), var(--accent-secondary)); border-radius: 3px; width: 0%; }
        .frame-counter { font-family: 'JetBrains Mono', monospace; font-size: 0.8rem; color: var(--text-secondary); min-width: 80px; text-align: right; }
        .speed-select { background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 6px; padding: 6px 10px; font-family: 'JetBrains Mono', monospace; font-size: 0.7rem; color: var(--text-primary); }
        .stats-grid { display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; margin-top: 16px; display: none; }
        .stats-grid.show { display: grid; }
        .stat-item { background: var(--bg-surface); border-radius: 10px; padding: 12px; text-align: center; }
        .stat-value { font-family: 'JetBrains Mono', monospace; font-size: 1.1rem; font-weight: 600; color: var(--accent-primary); }
        .stat-label { font-size: 0.7rem; color: var(--text-muted); margin-top: 2px; }
        .examples { margin-top: 16px; padding-top: 16px; border-top: 1px solid var(--border-color); }
        .examples h3 { font-size: 0.8rem; font-weight: 500; color: var(--text-secondary); margin-bottom: 10px; }
        .example-chips { display: flex; flex-wrap: wrap; gap: 6px; }
        .example-chip { padding: 6px 12px; background: var(--bg-surface); border: 1px solid var(--border-color); border-radius: 16px; font-size: 0.75rem; color: var(--text-secondary); cursor: pointer; transition: all 0.2s; }
        .example-chip:hover { background: rgba(99, 102, 241, 0.1); border-color: var(--accent-primary); color: var(--accent-primary); }
        .loading-overlay { position: absolute; top: 0; left: 0; width: 100%; height: 100%; background: rgba(10, 10, 15, 0.9); display: flex; flex-direction: column; align-items: center; justify-content: center; z-index: 100; opacity: 0; pointer-events: none; transition: opacity 0.3s; }
        .loading-overlay.active { opacity: 1; pointer-events: all; }
        .loading-spinner { width: 40px; height: 40px; border: 3px solid var(--border-color); border-top-color: var(--accent-primary); border-radius: 50%; animation: spin 1s linear infinite; }
        .loading-text { margin-top: 12px; color: var(--text-secondary); font-size: 0.85rem; }
        @media (max-width: 900px) { .main-layout { grid-template-columns: 1fr; } #viewer-container { min-height: 350px; } }
    </style>
</head>
<body>
    <div class="bg-pattern"></div>
    <div class="container">
        <header>
            <div class="logo">
                <div class="logo-icon">🤟</div>
                <h1>SOKE</h1>
            </div>
            <p class="subtitle">Real-time Text to Sign Language Generation</p>
            <div class="server-url-box">
                <div class="server-url-label">🔗 Server URL <span class="server-status disconnected" id="server-status">Disconnected</span></div>
                <input type="text" class="server-url-input" id="server-url" placeholder="https://xxxx-colab.googleusercontent.com" value="COLAB_URL_PLACEHOLDER">
            </div>
        </header>
        <div class="main-layout">
            <div class="panel">
                <div class="panel-header"><div class="panel-icon">✍️</div><h2>Input</h2></div>
                <div class="panel-body">
                    <div class="form-group">
                        <label for="text-input">Enter text to translate</label>
                        <textarea id="text-input" placeholder="Type a sentence...">Hello, how are you today?</textarea>
                    </div>
                    <div class="form-group">
                        <label for="language-select">Target Sign Language</label>
                        <select id="language-select">
                            <option value="how2sign">🇺🇸 American Sign Language (ASL)</option>
                            <option value="csl">🇨🇳 Chinese Sign Language (CSL)</option>
                            <option value="phoenix">🇩🇪 German Sign Language (DGS)</option>
                        </select>
                    </div>
                    <button id="generate-btn" class="generate-btn"><span class="btn-text">🚀 Generate Sign Language</span></button>
                    <div id="status-bar" class="status-bar"><span class="status-text"></span></div>
                    <div class="examples">
                        <h3>Try these examples:</h3>
                        <div class="example-chips">
                            <span class="example-chip" data-text="Hello, how are you?">Hello</span>
                            <span class="example-chip" data-text="Thank you for your help.">Thank you</span>
                            <span class="example-chip" data-text="Nice to meet you!">Nice to meet you</span>
                            <span class="example-chip" data-text="What is your name?">What's your name?</span>
                        </div>
                    </div>
                    <div class="stats-grid" id="stats-grid">
                        <div class="stat-item"><div class="stat-value" id="stat-frames">-</div><div class="stat-label">Frames</div></div>
                        <div class="stat-item"><div class="stat-value" id="stat-duration">-</div><div class="stat-label">Duration</div></div>
                        <div class="stat-item"><div class="stat-value" id="stat-time">-</div><div class="stat-label">Gen Time</div></div>
                    </div>
                </div>
            </div>
            <div class="panel" style="display: flex; flex-direction: column;">
                <div class="panel-header"><div class="panel-icon">👁️</div><h2>3D Visualization</h2></div>
                <div class="panel-body" style="padding: 0; flex: 1; display: flex; flex-direction: column;">
                    <div id="viewer-container">
                        <canvas id="three-canvas"></canvas>
                        <div class="viewer-placeholder" id="placeholder"><div class="icon">🧍</div><p>Enter text and click Generate</p></div>
                        <div class="loading-overlay" id="loading-overlay"><div class="loading-spinner"></div><div class="loading-text">Generating...</div></div>
                    </div>
                    <div class="playback-controls">
                        <button class="control-btn" id="play-btn" disabled>▶</button>
                        <button class="control-btn" id="pause-btn" disabled>⏸</button>
                        <button class="control-btn" id="reset-btn" disabled>↺</button>
                        <div class="timeline" id="timeline"><div class="timeline-progress" id="timeline-progress"></div></div>
                        <span class="frame-counter" id="frame-counter">0 / 0</span>
                        <select class="speed-select" id="speed-select"><option value="0.5">0.5x</option><option value="1" selected>1x</option><option value="2">2x</option></select>
                    </div>
                </div>
            </div>
        </div>
    </div>
    <script>
        let API_URL = '';
        const FPS = 20;
        let meshData = null, currentFrame = 0, totalFrames = 0, isPlaying = false, animationId = null, lastFrameTime = 0, playbackSpeed = 1;
        let scene, camera, renderer, controls, mesh = null, geometry = null;

        function updateServerUrl() {
            let url = document.getElementById('server-url').value.trim();
            if (!url) {
                showStatus('error', 'Please enter a server URL');
                return;
            }
            url = url.replace(/\\/$/, '');
            API_URL = url;
            localStorage.setItem('soke_server_url', url);
            checkServerStatus();
        }

        function checkServerStatus() {
            if (!API_URL) {
                document.getElementById('server-status').className = 'server-status disconnected';
                document.getElementById('server-status').textContent = 'Disconnected';
                return;
            }
            
            fetch(`${API_URL}/health`)
                .then(r => r.json())
                .then(d => {
                    if (d.status === 'healthy') {
                        document.getElementById('server-status').className = 'server-status connected';
                        document.getElementById('server-status').textContent = 'Connected ✓';
                        showStatus('success', 'Connected to SOKE API ✓');
                    }
                })
                .catch(() => {
                    document.getElementById('server-status').className = 'server-status disconnected';
                    document.getElementById('server-status').textContent = 'Connection Failed';
                    showStatus('error', 'Cannot connect to server. Check URL and ensure server is running.');
                });
        }

        function initThreeJS() {
            const container = document.getElementById('viewer-container');
            const canvas = document.getElementById('three-canvas');
            scene = new THREE.Scene();
            scene.background = new THREE.Color(0x12121a);
            camera = new THREE.PerspectiveCamera(45, container.clientWidth / container.clientHeight, 0.1, 100);
            camera.position.set(0, 0.5, 2.5);
            renderer = new THREE.WebGLRenderer({ canvas, antialias: true });
            renderer.setSize(container.clientWidth, container.clientHeight);
            renderer.setPixelRatio(window.devicePixelRatio);
            controls = new THREE.OrbitControls(camera, renderer.domElement);
            controls.enableDamping = true;
            controls.target.set(0, 0.5, 0);
            scene.add(new THREE.AmbientLight(0xffffff, 0.5));
            const light = new THREE.DirectionalLight(0xffffff, 0.8);
            light.position.set(2, 3, 2);
            scene.add(light);
            scene.add(new THREE.DirectionalLight(0x6366f1, 0.3).position.set(-2, 1, -1));
            const grid = new THREE.GridHelper(4, 20, 0x333344, 0x222233);
            grid.position.y = -0.5;
            scene.add(grid);
            window.addEventListener('resize', () => {
                camera.aspect = container.clientWidth / container.clientHeight;
                camera.updateProjectionMatrix();
                renderer.setSize(container.clientWidth, container.clientHeight);
            });
            (function animate() { requestAnimationFrame(animate); controls.update(); renderer.render(scene, camera); })();
        }

        function createMesh(vertices, faces) {
            if (mesh) { scene.remove(mesh); geometry.dispose(); }
            geometry = new THREE.BufferGeometry();
            geometry.setAttribute('position', new THREE.BufferAttribute(new Float32Array(vertices.flat()), 3));
            geometry.setIndex(new THREE.BufferAttribute(new Uint32Array(faces.flat()), 1));
            geometry.computeVertexNormals();
            mesh = new THREE.Mesh(geometry, new THREE.MeshPhongMaterial({ color: 0x6366f1, shininess: 30, side: THREE.DoubleSide }));
            scene.add(mesh);
            const box = new THREE.Box3().setFromObject(mesh);
            controls.target.copy(box.getCenter(new THREE.Vector3()));
        }

        function updateMesh(vertices) {
            if (!geometry) return;
            geometry.setAttribute('position', new THREE.BufferAttribute(new Float32Array(vertices.flat()), 3));
            geometry.computeVertexNormals();
        }

        function playAnimation() {
            if (!meshData || totalFrames === 0) return;
            isPlaying = true;
            document.getElementById('play-btn').disabled = true;
            document.getElementById('pause-btn').disabled = false;
            lastFrameTime = performance.now();
            function step(ts) {
                if (!isPlaying) return;
                if (ts - lastFrameTime >= 1000 / (FPS * playbackSpeed)) {
                    currentFrame = (currentFrame + 1) % totalFrames;
                    updateMesh(meshData.vertices[currentFrame]);
                    updateUI();
                    lastFrameTime = ts;
                }
                animationId = requestAnimationFrame(step);
            }
            animationId = requestAnimationFrame(step);
        }

        function pauseAnimation() {
            isPlaying = false;
            document.getElementById('play-btn').disabled = false;
            document.getElementById('pause-btn').disabled = true;
            if (animationId) cancelAnimationFrame(animationId);
        }

        function resetAnimation() {
            pauseAnimation();
            currentFrame = 0;
            if (meshData) updateMesh(meshData.vertices[0]);
            updateUI();
        }

        function updateUI() {
            document.getElementById('frame-counter').textContent = `${currentFrame + 1} / ${totalFrames}`;
            document.getElementById('timeline-progress').style.width = `${((currentFrame + 1) / totalFrames) * 100}%`;
        }

        function showStatus(type, msg) {
            const bar = document.getElementById('status-bar');
            bar.className = `status-bar show ${type}`;
            bar.querySelector('.status-text').textContent = msg;
            if (type === 'success') setTimeout(() => bar.classList.remove('show'), 4000);
        }

        async function generate() {
            if (!API_URL) {
                showStatus('error', 'Please enter server URL and press Enter');
                return;
            }
            
            const text = document.getElementById('text-input').value.trim();
            if (!text) { showStatus('error', 'Please enter text'); return; }
            const btn = document.getElementById('generate-btn');
            btn.disabled = true;
            btn.innerHTML = '<div class="spinner"></div><span>Generating...</span>';
            document.getElementById('placeholder').style.display = 'none';
            document.getElementById('loading-overlay').classList.add('active');
            showStatus('info', 'Generating sign language...');
            try {
                const res = await fetch(`${API_URL}/generate`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ text, sign_language: document.getElementById('language-select').value, fps: FPS, include_mesh: true })
                });
                const data = await res.json();
                if (!data.success) throw new Error(data.error || 'Failed');
                if (data.mesh_data?.vertices && data.mesh_data?.faces) {
                    meshData = data.mesh_data;
                    totalFrames = meshData.vertices.length;
                    currentFrame = 0;
                    createMesh(meshData.vertices[0], meshData.faces);
                    document.getElementById('play-btn').disabled = false;
                    document.getElementById('reset-btn').disabled = false;
                    document.getElementById('stats-grid').classList.add('show');
                    document.getElementById('stat-frames').textContent = data.num_frames;
                    document.getElementById('stat-duration').textContent = `${(data.num_frames / FPS).toFixed(2)}s`;
                    document.getElementById('stat-time').textContent = `${data.generation_time.toFixed(2)}s`;
                    updateUI();
                    showStatus('success', `Generated ${data.num_frames} frames!`);
                    playAnimation();
                } else throw new Error('No mesh data');
            } catch (e) {
                showStatus('error', `Error: ${e.message}`);
                document.getElementById('placeholder').style.display = 'flex';
            } finally {
                btn.disabled = false;
                btn.innerHTML = '<span class="btn-text">🚀 Generate Sign Language</span>';
                document.getElementById('loading-overlay').classList.remove('active');
            }
        }

        document.addEventListener('DOMContentLoaded', () => {
            initThreeJS();
            
            const savedUrl = localStorage.getItem('soke_server_url');
            const urlInput = document.getElementById('server-url');
            if (savedUrl && savedUrl !== 'COLAB_URL_PLACEHOLDER') {
                urlInput.value = savedUrl;
            }
            
            urlInput.addEventListener('change', updateServerUrl);
            urlInput.addEventListener('keydown', (e) => {
                if (e.key === 'Enter') {
                    updateServerUrl();
                }
            });
            
            updateServerUrl();
            
            document.getElementById('generate-btn').addEventListener('click', generate);
            document.getElementById('play-btn').addEventListener('click', playAnimation);
            document.getElementById('pause-btn').addEventListener('click', pauseAnimation);
            document.getElementById('reset-btn').addEventListener('click', resetAnimation);
            document.getElementById('timeline').addEventListener('click', e => {
                pauseAnimation();
                currentFrame = Math.floor((e.clientX - e.target.getBoundingClientRect().left) / e.target.offsetWidth * totalFrames);
                if (meshData) updateMesh(meshData.vertices[currentFrame]);
                updateUI();
            });
            document.getElementById('speed-select').addEventListener('change', e => { playbackSpeed = parseFloat(e.target.value); });
            document.querySelectorAll('.example-chip').forEach(c => c.addEventListener('click', () => { document.getElementById('text-input').value = c.dataset.text; }));
            document.getElementById('text-input').addEventListener('keydown', e => { if (e.key === 'Enter' && e.ctrlKey) generate(); });
        });
    </script>
</body>
</html>'''
    
    # Replace placeholder with actual URL
    html_content = html_template.replace('COLAB_URL_PLACEHOLDER', colab_url)
    
    # Save to file
    output_path = 'soke_ui.html'
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print("=" * 60)
    print("✅ STANDALONE UI FILE CREATED!")
    print("=" * 60)
    print(f"\n📁 File saved: {output_path}")
    print(f"🔗 Configured for: {colab_url}")
    print("\n" + "=" * 60)
    print("📥 HOW TO USE:")
    print("=" * 60)
    print("\n1. Download the file:")
    print("   - Click the folder icon (📁) on the left sidebar")
    print("   - Find 'soke_ui.html'")
    print("   - Right-click → Download")
    print("\n2. Open the HTML file:")
    print("   - Double-click the downloaded file")
    print("   - OR drag it into your browser")
    print("\n3. It will automatically connect to your Colab server!")
    print("   - No ngrok needed")
    print("   - Works from any device on any network")
    print("   - You can even share the file with others")
    print("\n💡 TIP: If you restart the Colab server, just run this cell")
    print("   again to generate a new HTML file with the updated URL.")
    print("\n" + "=" * 60)
    
    # Also trigger download
    from google.colab import files
    print("\n⬇️  Downloading file automatically...")
    files.download(output_path)
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nMake sure the server is running (cell 18) before generating the UI file.")



🌐 YOUR COLAB URL:

https://8081-gpu-t4-s-1u7ewy4rsqj7d-c.asia-southeast1-0.prod.colab.dev

📱 Open this URL in your browser!

If the page shows 'Connecting...', the server might not be running.
Go back and make sure cell 17 shows '✅ SERVER IS RUNNING!'


## 6. Test the API (Optional)


In [18]:
# Test the API
try:
    PORT
except NameError:
    PORT = 8080

import requests

print("Testing API...")
try:
    # First check health
    health = requests.get(f"http://localhost:{PORT}/health", timeout=5)
    if health.status_code != 200:
        print("❌ Server not responding. Run cell 17 first.")
    else:
        print("✓ Server is responding")

        # Now test generation
        response = requests.post(
            f"http://localhost:{PORT}/generate",
            json={
                "text": "Hello",
                "sign_language": "how2sign",
                "include_mesh": False
            },
            timeout=60
        )

        data = response.json()
        if data['success']:
            print(f"✅ Generation successful!")
            print(f"   Frames: {data['num_frames']}")
            print(f"   Time: {data['generation_time']:.3f}s")
        else:
            print(f"❌ Generation failed: {data.get('error')}")

except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to server. Make sure cell 17 ran successfully.")
except Exception as e:
    print(f"❌ Error: {e}")


Testing API...
✓ Server is responding
✅ Generation successful!
   Frames: 4
   Time: 14.858s


## 7. Keep Alive (Prevents Colab Timeout)

Run this cell to prevent Colab from disconnecting due to inactivity:


In [19]:
# Keep-alive loop - prevents Colab timeout
try:
    PORT
except NameError:
    PORT = 8080

import time
import requests

# First check if server is running
try:
    resp = requests.get(f"http://localhost:{PORT}/health", timeout=5)
    if resp.status_code != 200:
        print("❌ Server not running. Run cell 17 first.")
        raise SystemExit
except requests.exceptions.ConnectionError:
    print("❌ Server not running. Run cell 17 first.")
    raise SystemExit

print("🔄 Keep-alive started. Service will stay running.")
print("Press the stop button (⏹) in Colab to stop.\n")

try:
    while True:
        try:
            response = requests.get(f"http://localhost:{PORT}/health", timeout=10)
            status = response.json()
            print(f"\r[{time.strftime('%H:%M:%S')}] ✓ Running | GPU: {status['gpu_available']}", end="", flush=True)
        except:
            print(f"\r[{time.strftime('%H:%M:%S')}] ⚠ Checking...", end="", flush=True)
        time.sleep(60)
except KeyboardInterrupt:
    print("\n\n✓ Stopped.")
except SystemExit:
    pass


🔄 Keep-alive started. Service will stay running.
Press the stop button (⏹) in Colab to stop.

[23:26:35] ✓ Running | GPU: True

✓ Stopped.
